# SBTi Temperature Alignment tool examples
This notebook provides some examples on how to use the SBTi Temperature Alignment tool.

> This notebook is meant to run on Google Colab. If you're running the notebook locally, use the normal "1_getting_started.ipynb" notebook

## Install the SBTi Python module

In [ ]:
!pip install SBTi

In [ ]:
import SBTi
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.portfolio_coverage_tvp import PortfolioCoverageTVP
from SBTi.temperature_score import TemperatureScore, Scenario
from SBTi.target_validation import TargetValidation
import pandas as pd

## Download the dummy data provider

In [ ]:
import urllib.request
import os
os.mkdir("data")
urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/data/data_provider_example.xlsx", "data/data_provider_example.xlsx")
urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/example_portfolio.csv", "example_portfolio.csv")

#### Logging
The SBTi module uses the Python standard library logging utilities to send log messages. The log level can be changed according to the user's needs.

In [ ]:
import logging
root_logger = logging.getLogger()
root_logger.setLevel("INFO")

### Create a data provider
Data providers let you connect to the data source of your choice.

In [ ]:
provider = ExcelProvider(path="data/data_provider_example.xlsx")

### Load your portfolio
In our case the portfolio is stored as a CSV file. The portfolio should at least have an "id" (the identifier of the company) and a "proportion" (the weight of the company in your portfolio e.g. the value of the shares you hold) column.

In [ ]:
portfolio = pd.read_csv("example_portfolio.csv", encoding="iso-8859-1")

In [ ]:
portfolio.head(5)

To load the data from the data provider, we have to pass a list of dictionaries, containing a "company_name" and "company_id".

In [ ]:
companies = portfolio[["company_id", "company_name"]].to_dict(orient="records")

### Load the data
We'll call the data provider and ask it to provide us with the targets and the company data.

In [ ]:
company_data = SBTi.data.get_company_data([provider], companies)
target_data = SBTi.data.get_targets([provider], companies)
company_data = pd.merge(left=company_data,
                        right=portfolio[
                            [column
                                for column in portfolio.columns
                                if column not in ["company_name"]]],
                        how="left",
                        on=["company_id"])

### Validate the targets
We've now got our portfolio, amended with target data. Now we need to make sure that all the targets are valid (e.g. they haven't been reached yet and their end date isn't in the past).

In [ ]:
target_validation = TargetValidation(target_data, company_data)
portfolio_data = target_validation.target_validation()

### Calculate the temperature scores
In the amended portfolio you'll find your original portfolio, amended with the emissions and the temperature score.

In [ ]:
temperature_score = TemperatureScore()
amended_portfolio = temperature_score.calculate(portfolio_data)

In [ ]:
amended_portfolio.head(10)

### Calculate the aggregated temperature score
Calculate an aggregated temperature score. This can be done use different aggregation methods. Here we'll use the "Weighted Average Temperature Score". For more details, have a look at notebook 3 ("3_portfolio_aggregations.ipynb"). The termperature scores are calculated per time-frame/scope combination.

In [ ]:
def print_aggregations(aggregations):
    for time_frame, time_frame_values in aggregations.items():
        for scope, scope_values in time_frame_values.items():
            print("{} - {}: {} degrees celcius".format(time_frame, scope, scope_values["all"]["score"]))

In [ ]:
print_aggregations(temperature_score.aggregate_scores(amended_portfolio, PortfolioAggregationMethod.WATS, []))

### Portfolio coverage

Gives insights in the coverage of portfolio entities that has set GHG emissions reduction targets, STBi-validated or otherwise. Companies with SBTi status Approved have a score of 100%, all others have a score of 0%. 

This can be done use different aggregation methods. Here we'll use the "Weighted Average Temperature Score". For more details, have a look at notebook 3 ("3_portfolio_aggregations.ipynb").

In [ ]:
portfolio_coverage_tvp = PortfolioCoverageTVP()
coverage = portfolio_coverage_tvp.get_portfolio_coverage(amended_portfolio.copy(), PortfolioAggregationMethod.WATS)
round(coverage * 100) / 100